I have used a train malfunction dataset from NS, the principal passenger railway operator in the Netherlands, spanning the year 2019, and I have combined it with climate data from the same year, to predict how long it will take for a train malfunction to be resolved, and to help train users make more confident decisions against delays or cancellations.

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from time import time
import holidays 

In [ ]:
pd.options.display.max_columns = None

# Load Data

In [ ]:
data1 = pd.read_csv('W9 - Project - disruptions-2019.csv')

In [ ]:
data2 = pd.read_csv('W9 - Project - stations-2020-01.csv')

In [ ]:
data3 = pd.read_csv('W9 - Project - Climate Location Data.csv', sep = ';')

In [ ]:
data4 = pd.read_csv('W9 - Project - Climate Data.csv', sep = ';')

# data1

### data1 info
**rdt_id** - This is the ID that Driving the Trains uses for a fault. When you open an outage in the outage archive, you will see the ID processed in the URL of the outage page. For example, in this train disruption between Amsterdam South and Schiphol the ID 12345.

**ns_lines** - These are the routes that NS links to a disruption. In the case of the failure in the example, this is Schiphol-Almere C./Hilversum/Utrecht C. A problem with these routes (if you want to analyze the data) is that they are not standardized. This column is therefore less suitable for use for analyzes.

**rdt_lines** - These are the routes that Driving the Trains has linked to a disruption. This is always based on the list of routes of Driving the Trains, and the link is based on the stations where a fault is located. In the example, the disruption is between Amsterdam South and Schiphol. Driving the Trains then combines the Amersfoort-Schiphol, Lelystad-Schiphol and Utrecht-Schiphol routes. The trajectories are always linked in alphabetical order, separated by a comma.

**rdt_lines_id** - This is the ID of the routes that Driving the Trains has linked, which are separated by a comma.

**rdt_station_names** - Based on the link with the routes, Rijden de Trains also calculates which stations have been affected by a train disruption. This column lists the drive names of the affected drives, separated by a comma.

**rdt_station_codes** - These are the codes (abbreviations) of the stations involved, separated by a comma. The station codes can be found in the dataset with stations (see above).

**cause_nl** - This is the cause of a malfunction, in Dutch. If during a fault the cause is adjusted by NS, the last used cause is displayed in this column.

**cause_en** - The cause of the failure translated into English.

**statistical_cause_nl** - For statistical purposes, Driving the Trains also keeps track of a 'statistical cause'. When the cause of the failure changes, information about the actual cause of a failure is sometimes lost. If the failure cause is adjusted to a cause such as a previous failure, the original cause is shown in this column. For example, for this signal failure at Woerden, the last cause is 'a previous disturbance', but for statistics we assume the cause is major signal and points failure.

**statistical_cause_en** - The statistical cause, but in English.

**cause_group** - The group in which the fault is classified. These are the headings on the failure causes list, such as Accidents or Equipment Problems. In the column, the group is stored in English.

**start_time** - The time when the fault started.

**end_time** - The time at which the fault ended.

**duration_minutes** - The fault duration in minutes.

In [ ]:
data1.head()

### data1: splitting station codes into rows.
This way we will have the failures of each specific station.

In [ ]:
data1_fixed = (data1.set_index(data1.columns.drop('rdt_station_codes',1).tolist()).rdt_station_codes.str.split(',', expand=True).stack().reset_index().rename(columns={0:'rdt_station_codes'}).loc[:, data1.columns])

In [ ]:
data1_fixed.head()

In [ ]:
data1_fixed.columns

### data1: dropping columns.
Dropping columns that we immediately know will be of no use.

In [ ]:
data1_clean1 = data1_fixed.drop(['ns_lines','rdt_lines_id', 'cause_nl', 'statistical_cause_nl'], axis = 1)
data1_clean1.head()

### data1: preparing for merge with data2.
Creating empty columns which we will populate with data from data2.

In [ ]:
data1_clean1['uic'] = ''
data1_clean1['name_long'] = ''
data1_clean1['country'] = ''
data1_clean1['geo_lat'] = ''
data1_clean1['geo_lng'] = ''

In [ ]:
data1_clean1.head()

# data2

### data2 info
**id** - Internal ID of Driving the Trains for a station.

**code** - This is the NS / ProRail code (shortening) for a station. You often recognize the station name here, but there are some strange ducks in the bite (like most stations in The Hague). An abbreviation is max. 8 characters long.

**uic** - UIC code of a station. This is an international number that is unique per station. Dutch stations always start with 84.

**name_short** - Short station name. A large part of the station name is often omitted here.

**name_medium** - Medium station name.

**name_long** - Long station name. This field contains the full, official station name

**slug** - The 'slug' is part of the URL when you request departure or arrival times for the station. 

**country** - The country in which the station is located.

**type** - The kind of station. Possible values are (from most important to least important): mega station, intercity station junction, intercity station, express train station junction, express train station, stop train station junction, slow train station, optional station. Node as type means that you can transfer there. An optional Station is a station that is not normally used, except for events (such as Rotterdam Stadium, which is only used for football matches in De Kuip).

**geo_lat** - Latitude, part of the GPS coordinates.

**geo_lng** - Longitude, part of the GPS coordinates.

In [ ]:
data2.head()

In [ ]:
data2.columns

### data2: dropping columns.
Dropping columns that we immediately know will be of no use.

In [ ]:
data2_drop1 = data2.drop(['id','name_short','name_medium','slug','type'], axis = 1)
data2_drop1.head()

In [ ]:
data2_drop1.columns

### data2: prep for merge with data1.
Changing name of 'code' to 'rdt_station_codes' because we will merge the data from data2 with data1 on 'rdt_station_codes'.

In [ ]:
data2_drop1.columns = ['rdt_station_codes', 'uic', 'name_long', 'country', 'geo_lat', 'geo_lng']

In [ ]:
 data2_drop1.head()

In [ ]:
 data2_drop1.dtypes

In [ ]:
data2_drop1['uic'] = data2_drop1['uic'].astype('Int64')

In [ ]:
 data2_drop1.dtypes

# data1 - data2 Merge

### data1_clean1: cleaning strings.
We have to clean 'data1_clean1['rdt_station_codes']' to avoid small errors in the string that will lead to an incorrect match with data2 and to populating data1_clean1 with NaN values.

In [ ]:
data1_clean1['rdt_station_codes'] = data1_clean1['rdt_station_codes'].apply(lambda x : x.strip().upper()) 

### data1 - data2 merge.
populating data1 with values from data2, based on coincidence with 'rdt_station_codes'.

In [ ]:
df = (data1_clean1.merge(data2_drop1, on='rdt_station_codes', how='left', suffixes=('_',''))
     .drop(['uic_', 'name_long_', 'country_', 'geo_lat_', 'geo_lng_'], axis=1)
     .reindex(columns=data1_clean1.columns))

In [ ]:
df.head(50)

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.DataFrame(pd.to_datetime(df['start_time']).dt.date)
df['time_start'] = pd.DataFrame(pd.to_datetime(df['start_time']).dt.time)

df['date2'] = pd.DataFrame(pd.to_datetime(df['end_time']).dt.date)
df['time_end'] = pd.DataFrame(pd.to_datetime(df['end_time']).dt.time)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df_ns = df.drop(['start_time','end_time','date2'], axis=1)
df_ns.head()

In [ ]:
df_ns.dtypes

### changing dtypes of train data

In [ ]:
df_ns[['rdt_id', 'uic']] = df_ns[['rdt_id', 'uic']].astype('object')
df_ns['date'] = pd.to_datetime(df_ns['date'])

In [ ]:
df_ns.dtypes

In [ ]:
df_ns.head()

# data3: location data for climate measuring stations

In [ ]:
data3 = data3.drop(['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis = 1)

In [ ]:
data3.head()

#### Because of the size of The Netherlands, we will choose one location from which to read the weather data. De Bilt (STN = 260), in Utrecht, is the weather station from which the main weather information is retreived on a national scale. De Bilt is the chosen location.

In [ ]:
debilt = data3['STN'] == 260

In [ ]:
data_debilt = data3[debilt]
data_debilt

# data4: Climate Data

### data4 info
### YYYYMMDD = Date (YYYY=year MM=month DD=day); 
**FHX**  = Highest hourly mean wind speed (in 0.1 m/s); 

**FHXH**     = Hour slot in which FHX is measured; 

**FHN**      = Lowest hourly mean wind speed (in 0.1 m/s); 

**FHNH**     = Hour slot in which FHN is measured; 

**TN**       = Minimum temperature (in 0.1 degrees Celsius); 

**TNH**      = Hour slot in which TN is measured; 

**TX**       = Maximum temperature (in 0.1 degrees Celsius); 

**TXH**      = Hourly slot in which TX is measured; 

**SQ**       = Sunshine duration (in 0.1 hours) calculated from the global radiation (-1 for <0.05 hours); 

**SP**       = Percentage of the longest possible sunshine duration; 

**Q**        = Global radiation (in J/cm2); 

**DR**       = Duration of precipitation (in 0.1 uur); 

**RH**       = Several times the precipitation (in 0.1 mm) (-1 for <0.05 mm); 

**RHX**      = Highest hourly sum of the precipitation (in 0.1 mm) (-1 for <0.05 mm); 

**PG**       = 24-hour average air pressure converted to sea level (in 0.1 hPa) calculated from 24 hour values; 

**VVN**      = Minimum sight occurred; 0: <100m, 1: 100-200m, 2: 200-300m, ..., 49: 4900-5000m, 50: 5-6km, 56: 6-7km, 57: 7-8 km, ..., 79: 29-30 km, 80: 30-35 km, 81: 35-40 km, ..., 89:> 70 km)

**VVX**      = Maximum visibility occurred; 0: <100m, 1: 100-200m, 2: 200-300m, ..., 49: 4900-5000m, 50: 5-6km, 56: 6-7km, 57: 7-8 km, ..., 79: 29-30 km, 80: 30-35 km, 81: 35-40 km, ..., 89:> 70 km)

**NG**       = 24-hour average cloud cover (degree of cover of the upper air in eighths, 9 = upper air invisible); 

**UG**       = 24-hour average relative humidity (in percent); 

**UX**       = Maximum relative humidity (in percent); 

**UXH**      = Hourly slot in which UX is measured; 

**UN**       = Minimum relative humidity (in percent); 

**UNH**      = Hour period in which UN was measured; 

In [ ]:
data4.tail()

### data4: dropping columns.
Dropping columns that we immediately know will be of no use.

In [ ]:
data4.columns

In [ ]:
data4 = data4.drop([' FHXH','  FHN',' FHNH','  TNH','  TXH','   SQ','   SP','   DR','   RH','   PG','   NG','   UX','  UXH','   UN','  UNH'], axis = 1)
data4.head()

### data4 filtering to get only the readings from De Bilt (Utrecht)

In [ ]:
data_weather = data4['# STN'] == 260

In [ ]:
data_weather_debilt = data4[data_weather]
data_weather_debilt.head()

### data4 change column names

In [ ]:
data_weather_debilt.columns

In [ ]:
data_weather_debilt.columns = ['weather_station_code','date','highest_mean_wind_speed','min_temp','max_temp','radiation','rain','min_visibility', 'max_visibility', 'avg_relative_humidity']
data_weather_debilt.head()

### Changing dtypes of climate data

In [ ]:
data_weather_debilt.dtypes

In [ ]:
data_weather_debilt.columns

In [ ]:
data_weather_debilt[['highest_mean_wind_speed','min_temp','max_temp', 'radiation', 'rain', 'min_visibility','max_visibility', 'avg_relative_humidity']] = data_weather_debilt[['highest_mean_wind_speed','min_temp','max_temp', 'radiation', 'rain', 'min_visibility','max_visibility', 'avg_relative_humidity']].astype('int64')
data_weather_debilt['date'] = pd.to_datetime(data_weather_debilt['date'].astype('str'), format = '%Y-%m-%d')

In [ ]:
data_weather_debilt.dtypes

In [ ]:
data_weather_debilt.head()

### data4: computing the mean temperature and visibility

In [ ]:
data_weather_debilt['avg_temp'] = data_weather_debilt[['min_temp','max_temp']].mean(axis=1)

In [ ]:
data_weather_debilt['avg_visibility'] = data_weather_debilt[['min_visibility','max_visibility']].mean(axis=1)

### data4: Dropping weather_station_code column, which is irrelevant after we have already saved a filtered dataframe.

In [ ]:
data_weather_debilt = data_weather_debilt.drop(['weather_station_code','min_temp','max_temp','min_visibility','max_visibility'], axis=1)
data_weather_debilt.head()

# Merge train data (df_ns) and climate data (data_weather_debilt)

### Create empty columns in df_ns to populate with data_weather_debilt

In [ ]:
df_ns['highest_mean_wind_speed'] = ''
df_ns['radiation'] = ''
df_ns['rain'] = ''
df_ns['avg_relative_humidity'] = ''
df_ns['avg_temp'] = ''
df_ns['avg_visibility'] = ''

In [ ]:
data_merge = (df_ns.merge(data_weather_debilt, on='date', how='left', suffixes=('_',''))
     .drop(['highest_mean_wind_speed_', 'radiation_', 'rain_', 'avg_relative_humidity_', 'avg_temp_','avg_visibility_'], axis=1)
     .reindex(columns=df_ns.columns))

In [ ]:
data_merge.head()

In [ ]:
data_merge.dtypes

# Final cleanup.
I will drop some extra unnecessary columns and check for NaN values. I want to have a clean dataframe in order to save as csv and later use for data exploration and model building.

### data_merge: Renaming and re-arranging columns

In [ ]:
data_merge.head()

In [ ]:
data_merge.columns

In [ ]:
data = data_merge.drop(['rdt_id','rdt_lines','rdt_station_names','cause_en','country'], axis=1)
data.head()

In [ ]:
data = data.rename(columns={'rdt_station_codes':'station_code', 'statistical_cause_en':'malfunction_cause', 'cause_group':'malfunction_cause_group','uic':'station_id','name_long':'station_name','geo_lat':'latitude','geo_lng':'longitude','rain':'rain_mm'})

In [ ]:
data = data[['station_id','station_code','station_name','latitude','longitude','date','time_start','time_end','malfunction_cause','malfunction_cause_group','duration_minutes','avg_temp','rain_mm','highest_mean_wind_speed','avg_relative_humidity','radiation','avg_visibility']]

In [ ]:
data.head()

### data: NaN Values

In [ ]:
data.shape

In [ ]:
data.isnull().sum()/len(data_merge)*100

The percentage of null values is extremely low: we will proceed to drop.

In [ ]:
data = data.dropna()

In [ ]:
data.shape

In [ ]:
data.head()

### Save data_merge as csv

In [ ]:
data.to_csv(r'C:\Users\yelam\OneDrive\Escritorio\IRONHACK\WEEK 9 - Project\W9 - Project - project csv\W9 - ProjectData_1.csv', index = False, header=True)

# New database exchanging date for month and day of the week.
### data_2: Create column weekday and month
I can later on proceed to drop the date column for the purpose of the project.

In [ ]:
data_2 = data.copy()

In [ ]:
nl_holidays = holidays.Netherlands(years = 2019)
nl_holidays

In [ ]:
data_2['month'] = data_2['date'].dt.month
data_2['day_of_week'] = data_2['date'].dt.day_name()
data_2['holiday'] = data_2['date'].isin(nl_holidays)

data_2.head()

In [ ]:
data_2.columns

In [ ]:
data_2 = data_2[['station_id', 'station_code', 'station_name', 'latitude', 'longitude',
       'date','time_start', 'time_end','month','day_of_week', 'holiday','malfunction_cause', 'malfunction_cause_group',
       'duration_minutes', 'avg_temp', 'rain_mm', 'highest_mean_wind_speed',
       'avg_relative_humidity', 'radiation', 'avg_visibility', ]]

In [ ]:
data_2.head()

In [ ]:
data_2['month'] = data_2['month'].astype('object')

In [ ]:
data_2.to_csv(r'C:\Users\yelam\OneDrive\Escritorio\IRONHACK\WEEK 9 - Project\W9 - Project - project csv\W9 - ProjectData_2.csv', index = False, header=True)